# 🛠️ Google ADK Avanzado: Dominando las Herramientas (Tools)
## Tutorial Práctico para Google Colab

---

## 📋 Índice
1. **Configuración Inicial****
2. **¿Qué son las Herramientas en ADK?**
3. **Herramientas Preconstruidas**
4. **Creando Herramientas Personalizadas**
5. **Buenas practicas**
6. **Casos de Uso Avanzados**


## 1. Configuración Inicial

### Instalación del ADK

In [ ]:
# Instalar Google ADK
!pip install -q google-adk==1.4.2

In [1]:
# Importaciones necesarias
import os
from google.adk.agents import LlmAgent
from google.adk.tools import google_search
from google.genai import types
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from getpass import getpass

#### Opcion 1: Ingresalas directamente

In [ ]:
# Solicitar API Key de forma segura
if 'GOOGLE_API_KEY' not in os.environ:
    print("🔑 Por favor, ingresa tu Google API Key:")
    api_key = getpass("API Key: ")
    os.environ['GOOGLE_API_KEY'] = api_key
    os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'FALSE'
    print("\n✅ API Key configurada correctamente")
else:
    print("✅ API Key ya configurada")

# Verificar que las variables estén configuradas
print(f"\n📋 Variables de entorno configuradas:")
print(f"   - GOOGLE_API_KEY: {'✓' if os.environ.get('GOOGLE_API_KEY') else '✗'}")
print(f"   - GOOGLE_GENAI_USE_VERTEXAI: {os.environ.get('GOOGLE_GENAI_USE_VERTEXAI', 'No configurado')}")

print("✅ ADK instalado y configurado correctamente!")

#### Opcion 2: Cargar por Dotenv

In [2]:
from dotenv import load_dotenv
# Cargar variables de entorno desde .env si existe
load_dotenv(override=True)

True

## 2. ¿Qué son las Herramientas en ADK?

Las **herramientas (tools)** son capacidades o funciones que un agente puede invocar para:
- 🔍 Buscar información
- 🧮 Realizar cálculos
- 🌐 Interactuar con APIs
- 📊 Procesar datos
- ⚙️ Ejecutar acciones específicas

### Conceptos Clave

In [3]:
# Visualizar el concepto de herramientas
print("""
🤖 AGENTE LLM
    ↓
📦 CAJA DE HERRAMIENTAS
    ├── 🔍 Búsqueda Google
    ├── 🧮 Calculadora
    ├── 📊 Base de Datos
    └── 🌐 APIs Externas
    ↓
🌍 ACCIÓN EN EL MUNDO REAL
""")


🤖 AGENTE LLM
    ↓
📦 CAJA DE HERRAMIENTAS
    ├── 🔍 Búsqueda Google
    ├── 🧮 Calculadora
    ├── 📊 Base de Datos
    └── 🌐 APIs Externas
    ↓
🌍 ACCIÓN EN EL MUNDO REAL



## 3. Herramientas Preconstruidas

### Ejemplo: GoogleSearchTool

##### Nuestra funcion de inferencia

In [4]:
async def call_agent_async(query: str, runner, user_id, session_id):
    """Envía una consulta al agente e imprime la respuesta final."""
    print(f"\n>>> Consulta del usuario: {query}")

    # Prepara el mensaje del usuario en el formato de ADK
    content = types.Content(role='user', parts=[types.Part(text=query)])

    final_response_text = "El agente no produjo una respuesta final." # Valor por defecto

    # Concepto clave: run_async ejecuta la lógica del agente y genera eventos.
    # Iteramos a través de los eventos para encontrar la respuesta final.
    async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
        # Puedes descomentar la línea de abajo para ver *todos* los eventos durante la ejecución
        # print(f"  [Evento] Autor: {event.author}, Tipo: {type(event).__name__}, Final: {event.is_final_response()}, Contenido: {event.content}")

        # Concepto clave: is_final_response() marca el mensaje que concluye el turno.
        if event.is_final_response():
            if event.content and event.content.parts:
                # Se asume que la respuesta de texto está en la primera parte
                final_response_text = event.content.parts[0].text
            elif event.actions and event.actions.escalate: # Maneja posibles errores/escalamientos
                final_response_text = f"El agente escaló: {event.error_message or 'Sin mensaje específico.'}"
            # Agrega más validaciones aquí si es necesario (por ejemplo, códigos de error específicos)
            break # Deja de procesar eventos una vez encontrada la respuesta final

    print(f"<<< Respuesta del agente: {final_response_text}")


In [5]:
# Crear un agente con la herramienta de búsqueda de Google
agente_search = LlmAgent(
    name="InvestigadorGoogle",
    model="gemini-2.5-flash",
    description="Un agente que usa búsqueda de Google para responder preguntas actuales",
    tools=[google_search],  # Herramienta preconstruida
    instruction=(
        "Eres un investigador experto. "
        "Usa la búsqueda de Google para encontrar información actualizada. "
        "Cita tus fuentes cuando sea posible."
    )
)

print("✅ Agente investigador creado con GoogleSearchTool")

✅ Agente investigador creado con GoogleSearchTool


#### Probar el agente investigador

In [6]:
# Concepto clave: SessionService almacena el historial y estado de la conversación.
# InMemorySessionService es un almacenamiento simple y no persistente para este tutorial.
session_service = InMemorySessionService()

# Definir constantes para identificar el contexto de la interacción
APP_NAME = "agente_search_tool"
USER_ID = "user_1"
SESSION_ID = "session_001" # Usando un ID fijo por simplicidad

# Crear la sesión específica donde ocurrirá la conversación
session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)
# Runner: Este es el componente principal que gestiona la interacción con el agente.
runner = Runner(agent=agente_search,
                app_name=APP_NAME,
                session_service=session_service)



In [7]:
await call_agent_async("Como se llama el nuevo papa, para 2025",
                        runner=runner,
                        user_id=USER_ID,
                        session_id=SESSION_ID)


>>> Consulta del usuario: Como se llama el nuevo papa, para 2025
<<< Respuesta del agente: El nuevo Papa para 2025 se llama León XIV, cuyo nombre secular es Robert Francis Prevost. Fue elegido el 8 de mayo de 2025, sucediendo al Papa Francisco, quien falleció el 21 de abril de 2025.

Robert Francis Prevost nació el 14 de septiembre de 1955 en Chicago, Illinois, Estados Unidos. Es el Papa número 267 de la Iglesia Católica y el primer pontífice estadounidense, además de tener nacionalidad peruana. Antes de su elección, se desempeñó como Prefecto del Dicasterio para los Obispos y presidente de la Pontificia Comisión para América Latina (2023-2025). Su elección se produjo tras un cónclave que duró dos días y en el que participaron 133 cardenales electores.


### Ejemplo: BuiltInCodeExecutor para ejecución de codigo

In [8]:
from google.adk.code_executors import BuiltInCodeExecutor

AGENT_NAME = "calculator_agent"
APP_NAME = "calculator"
USER_ID = "user1234"
SESSION_ID = "session_code_exec_async"
GEMINI_MODEL = "gemini-2.0-flash"

code_agent = LlmAgent(
    name=AGENT_NAME,
    model=GEMINI_MODEL,
    code_executor=BuiltInCodeExecutor(),  # habilita ejecución de código
    instruction="""Eres un agente calculadora.
    Cuando se te proporcione una expresión matemática, escribe y ejecuta código Python para calcular el resultado.
    Devuelve únicamente el resultado numérico final como texto plano, sin formato markdown ni bloques de código.
    """,
    description="Ejecuta código Python para realizar cálculos.",
)

# Session and Runner
session_service = InMemorySessionService()
session = await session_service.create_session(
    app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID
)
runner = Runner(agent=code_agent, app_name=APP_NAME, session_service=session_service)

In [9]:
# Función asincrónica para enviar una consulta al agente y procesar los eventos que devuelve
async def call_agent_async_code(query, runner, user_id, session_id):
    # Crear el mensaje con el texto del usuario en el formato requerido por ADK
    content = types.Content(role="user", parts=[types.Part(text=query)])
    print(f"\n--- Ejecutando Consulta: {query} ---")

    # Variable para almacenar la respuesta final del agente
    final_response_text = "No se capturó una respuesta final de texto."

    # Inicia el ciclo asincrónico para procesar cada evento emitido por el agente
    async for event in runner.run_async(
        user_id=user_id, session_id=session_id, new_message=content
    ):
        print(f"ID del Evento: {event.id}, Autor: {event.author}")

        # --- Verificar si el evento contiene partes específicas como código o resultados ---
        has_specific_part = False
        if event.content and event.content.parts:
            for part in event.content.parts:  # Iterar por todas las partes del contenido
                if part.executable_code:
                    # Si hay código ejecutable generado por el agente, lo imprimimos
                    print(
                        f"  Debug: Código generado por el agente:\n```python\n{part.executable_code.code}\n```"
                    )
                    has_specific_part = True
                elif part.code_execution_result:
                    # Si hay resultado de ejecución de código, mostrar el resultado
                    print(
                        f"  Debug: Resultado de ejecución de código: {part.code_execution_result.outcome} - Salida:\n{part.code_execution_result.output}"
                    )
                    has_specific_part = True
                elif part.text and not part.text.isspace():
                    # Si hay texto plano, lo mostramos (no se considera parte "específica")
                    print(f"  Texto: '{part.text.strip()}'")
                    # No marcamos `has_specific_part = True` aquí para no interferir con la lógica de respuesta final

        # --- Verificar si es una respuesta final (después de manejar partes específicas) ---
        # Solo consideramos esta respuesta como final si no hubo partes específicas antes
        if not has_specific_part and event.is_final_response():
            if (
                event.content
                and event.content.parts
                and event.content.parts[0].text
            ):
                final_response_text = event.content.parts[0].text.strip()
                print(f"==> Respuesta Final del Agente: {final_response_text}")
            else:
                print("==> Respuesta Final del Agente: [Sin contenido de texto en el evento final]")

    # Mensaje final de cierre
    print("-" * 30)


#### Test 1

In [10]:
await call_agent_async_code("Calcula el valor de ((5 + 7 + 10) * 12) elevado a la 2", runner, USER_ID, SESSION_ID)


--- Ejecutando Consulta: Calcula el valor de ((5 + 7 + 10) * 12) elevado a la 2 ---


ID del Evento: 0lMczDCk, Autor: calculator_agent
  Debug: Código generado por el agente:
```python
result = ((5 + 7 + 10) * 12)**2
print(result)

```
  Debug: Resultado de ejecución de código: OUTCOME_OK - Salida:
69696

  Texto: '69696'
------------------------------


#### Test 2

In [11]:
await call_agent_async_code("Cual es el factorial de 10?", runner, USER_ID, SESSION_ID)


--- Ejecutando Consulta: Cual es el factorial de 10? ---


ID del Evento: nRVyAYYt, Autor: calculator_agent
  Debug: Código generado por el agente:
```python
import math
print(math.factorial(10))

```
  Debug: Resultado de ejecución de código: OUTCOME_OK - Salida:
3628800

  Texto: '3628800'
------------------------------


## 4. Creando Herramientas Personalizadas

### 🧮 Herramienta 1: Calculadora Simple

In [12]:
def sumar_numeros(a: int, b: int) -> dict:
    """
    Suma dos números enteros y devuelve el resultado de forma estructurada.

    Usa esta herramienta cuando necesites realizar una suma de dos valores enteros.
    
    Args:
        a (int): Primer número a sumar.
        b (int): Segundo número a sumar.

    Returns:
        dict: Un diccionario con las siguientes claves:
            - 'status' (str): "success" si la operación fue exitosa.
            - 'result' (int): Resultado de la suma (a + b).
            - 'operation' (str): Descripción de la operación realizada.
    """
    try:
        print(f"🧮 Herramienta sumar_numeros llamada con a={a}, b={b}")
        resultado = a + b
        return {
            "status": "success",
            "result": resultado,
            "operation": f"Suma de {a} + {b}"
        }
    except Exception as e:
        return {
            "status": "error",
            "error_message": f"Ocurrió un error al sumar los números: {str(e)}"
        }


### 🛍️ Herramienta 2: Búsqueda de Productos

In [13]:
import json

def buscar_producto_por_nombre(nombre_producto: str) -> dict:
    """
    Busca un producto por su nombre en el catálogo y devuelve un diccionario con sus detalles.

    Usa esta herramienta si el usuario solicita información de un producto específico.

    Args:
        nombre_producto (str): Nombre del producto a buscar (no sensible a mayúsculas).

    Returns:
        dict: Diccionario con los siguientes posibles campos:
            - 'status' (str): "success" si se encontró el producto, "error" si no.
            - 'product' (dict, opcional): Detalles del producto si fue encontrado.
            - 'error_message' (str, opcional): Mensaje explicativo si no se encontró el producto.
    """
    print(f"🛍️ Buscando producto: '{nombre_producto}'")

    # Simulación de base de datos
    productos_db = {
        "laptop gamer": {
            "id": "LPG001",
            "nombre": "Laptop Gamer Pro",
            "precio": 1500,
            "stock": 10,
            "características": ["RTX 4070", "32GB RAM", "1TB SSD"]
        },
        "teclado mecánico": {
            "id": "TEC005",
            "nombre": "Teclado Mecánico RGB",
            "precio": 120,
            "stock": 25,
            "características": ["Switches Cherry MX", "RGB", "TKL"]
        },
        "monitor 4k": {
            "id": "MON003",
            "nombre": "Monitor 4K HDR",
            "precio": 400,
            "stock": 5,
            "características": ["27 pulgadas", "144Hz", "HDR10"]
        }
    }

    producto = productos_db.get(nombre_producto.lower())
    
    if producto:
        return {
            "status": "success",
            "product": producto
        }
    else:
        return {
            "status": "error",
            "error_message": f"Producto '{nombre_producto}' no encontrado en el catálogo."
        }


### 📈 Herramienta 3: Cálculo de Porcentaje

In [14]:
def calcular_porcentaje(valor: float, porcentaje: float) -> dict:
    """
    Calcula el porcentaje de un valor base.

    Usa esta herramienta para calcular descuentos, aumentos, comisiones, proporciones u otros valores relativos a un porcentaje.

    Args:
        valor (float): Valor base sobre el cual se desea calcular el porcentaje.
        porcentaje (float): Porcentaje a aplicar (ej. 15 para 15%).

    Returns:
        dict: Diccionario con las siguientes claves:
            - 'status' (str): "success" si el cálculo fue exitoso.
            - 'result' (float): Valor calculado, redondeado a dos decimales.
            - 'details' (str): Explicación del cálculo realizado.
    """
    try:
        print(f"📈 Calculando {porcentaje}% de {valor}")
        resultado = round((valor * porcentaje) / 100, 2)
        return {
            "status": "success",
            "result": resultado,
            "details": f"{porcentaje}% de {valor} es {resultado}"
        }
    except Exception as e:
        return {
            "status": "error",
            "error_message": f"No se pudo calcular el porcentaje: {str(e)}"
        }


### Crear un agente con herramientas personalizadas

In [15]:
# Crear agente con nuestras herramientas personalizadas
agente_fuction_tool = LlmAgent(
    name="AgenteFunction",
    model="gemini-2.0-flash",
    description="Un agente que util que usa las herramientas personalizadas para cálculos y búsquedas",
    tools=[sumar_numeros, calcular_porcentaje, buscar_producto_por_nombre],  # Nuestras herramientas
    generate_content_config=types.GenerateContentConfig(
        temperature=0.1,  # Baja temperatura para cálculos precisos
        max_output_tokens=300
    ),
    instruction=(
        "Eres un asistente preciso. "
        "Ayuda al usuario usando las herramientas disponibles. "
    )
)

print("✅ Agente calculador creado con herramientas personalizadas")

✅ Agente calculador creado con herramientas personalizadas


In [16]:
# Concepto clave: SessionService almacena el historial y estado de la conversación.
# InMemorySessionService es un almacenamiento simple y no persistente para este tutorial.
session_service = InMemorySessionService()

# Definir constantes para identificar el contexto de la interacción
APP_NAME = "agente_function_tool"
USER_ID = "user_1"
SESSION_ID = "session_001" # Usando un ID fijo por simplicidad

# Crear la sesión específica donde ocurrirá la conversación
session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)
# Runner: Este es el componente principal que gestiona la interacción con el agente.
runner = Runner(agent=agente_fuction_tool,
                app_name=APP_NAME,
                session_service=session_service)



### Probar el agente de funciones

In [17]:
await call_agent_async("¿Cuánto es 15% de 1200?", runner=runner, user_id=USER_ID, session_id=SESSION_ID)


>>> Consulta del usuario: ¿Cuánto es 15% de 1200?


📈 Calculando 15% de 1200
<<< Respuesta del agente: El 15% de 1200 es 180.


In [18]:
await call_agent_async("Cuales son las caracteristicas y el precio del teclado mecánico?", runner=runner, user_id=USER_ID, session_id=SESSION_ID)


>>> Consulta del usuario: Cuales son las caracteristicas y el precio del teclado mecánico?


🛍️ Buscando producto: 'teclado mecánico'
<<< Respuesta del agente: El teclado mecánico RGB tiene switches Cherry MX, iluminación RGB, es TKL y cuesta 120.


In [19]:
await call_agent_async("Cuanto es 10 + 9991", runner=runner, user_id=USER_ID, session_id=SESSION_ID)


>>> Consulta del usuario: Cuanto es 10 + 9991


🧮 Herramienta sumar_numeros llamada con a=10, b=9991
<<< Respuesta del agente: 10 + 9991 es 10001.


## 5. Buenas practicas


### Importancia de los Type Hints y Docstrings

In [20]:
# Ejemplo de herramienta MAL definida (NO HACER ESTO)

def herramienta_mal_definida(x, y):  # ❌ No tiene type hints
    """Hace algo"""  # ❌ Descripción vaga, sin contexto útil para el agente
    return x + y  # ❌ No se sabe qué tipo de valores espera ni qué hace realmente


# Ejemplo de herramienta BIEN definida (HACER ESTO)

from typing import Dict

def convertir_temperatura(celsius: float) -> Dict[str, float]:
    """
    Convierte una temperatura en grados Celsius a Fahrenheit y Kelvin.

    Usa esta herramienta cuando el usuario necesite transformar una temperatura desde la escala Celsius
    a otras escalas comunes como Fahrenheit o Kelvin.

    Args:
        celsius (float): Temperatura en grados Celsius.

    Returns:
        dict: Diccionario con las siguientes claves:
            - 'celsius': Valor original en Celsius.
            - 'fahrenheit': Valor convertido a Fahrenheit.
            - 'kelvin': Valor convertido a Kelvin.
    """
    fahrenheit = (celsius * 9/5) + 32
    kelvin = celsius + 273.15

    return {
        "status": "success",
        "celsius": celsius,
        "fahrenheit": round(fahrenheit, 2),
        "kelvin": round(kelvin, 2)
    }

print("✅ Ejemplo de herramienta bien definida creada")


✅ Ejemplo de herramienta bien definida creada


## 6. Casos de Uso Avanzados {#casos-avanzados}

### 🎯 Herramienta con Estado y Contexto

In [21]:
carrito_compras: list[dict] = []  # Estado simulado del carrito

def agregar_al_carrito(producto: str, cantidad: int) -> dict:
    """
    Agrega un producto válido al carrito de compras.

    Usa esta herramienta cuando el usuario desee agregar un producto específico con cierta cantidad.

    Args:
        producto (str): Nombre del producto a agregar (no sensible a mayúsculas).
        cantidad (int): Cantidad deseada del producto.

    Returns:
        dict: Diccionario con:
            - 'status': "success" o "error"
            - 'message': Confirmación o motivo del fallo
            - 'current_cart_size': Número total de ítems en el carrito tras la acción (si fue exitosa)
    """
    global carrito_compras

    carrito_compras.append({
        "producto": producto.lower(),
        "cantidad": cantidad
    })

    total_items = sum(item["cantidad"] for item in carrito_compras)

    return {
        "status": "success",
        "message": f"Agregado {cantidad}x '{producto}' al carrito.",
        "current_cart_size": total_items
    }

def ver_carrito() -> dict:
    """
    Devuelve el contenido actual del carrito de compras.

    Usa esta herramienta si el usuario solicita revisar qué productos ha agregado al carrito.

    Returns:
        dict: Diccionario con:
            - 'status': "success" o "empty"
            - 'items': Lista de productos en el carrito (si hay)
            - 'total_items': Número total de unidades (si hay)
            - 'message': Texto breve sobre el estado del carrito
    """
    global carrito_compras

    if not carrito_compras:
        return {
            "status": "empty",
            "items": [],
            "total_items": 0,
            "message": "El carrito está vacío."
        }

    total_items = sum(item["cantidad"] for item in carrito_compras)

    return {
        "status": "success",
        "items": carrito_compras,
        "total_items": total_items,
        "message": f"Hay {len(carrito_compras)} productos en el carrito, totalizando {total_items} unidades."
    }


### Crear agente de comercio electrónico

In [22]:
# Agente especializado en e-commerce
agente_ecommerce = LlmAgent(
    name="AgenteEcommerce",
    model="gemini-2.5-flash",
    description="Asistente de compras online",
    tools=[
        buscar_producto_por_nombre,
        agregar_al_carrito,
        ver_carrito,
        calcular_porcentaje  # Para descuentos
    ],
    generate_content_config=types.GenerateContentConfig(
        temperature=0.2,
        max_output_tokens=400
    ),
    instruction=(
        "Eres un asistente de compras amigable y servicial. "
        "Ayuda a los usuarios a encontrar productos, agregarlos al carrito "
        "y calcular descuentos cuando sea necesario. "
        "Sé proactivo sugiriendo productos relacionados."
    )
)

print("✅ Agente de e-commerce creado")

✅ Agente de e-commerce creado


### Probar flujo de compra

In [23]:
# Concepto clave: SessionService almacena el historial y estado de la conversación.
# InMemorySessionService es un almacenamiento simple y no persistente para este tutorial.
session_service = InMemorySessionService()

# Definir constantes para identificar el contexto de la interacción
APP_NAME = "agente_ecommerce"
USER_ID = "user_4"
SESSION_ID = "004" # Usando un ID fijo por simplicidad

# Crear la sesión específica donde ocurrirá la conversación
session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)
# Runner: Este es el componente principal que gestiona la interacción con el agente.
runner = Runner(agent=agente_ecommerce,
                app_name=APP_NAME,
                session_service=session_service)



In [24]:
# Simular un flujo de compra
flujo_compra = [
    "Muéstrame información sobre la laptop gamer",
    "Agrega 1 Laptop Gamer Pro al carrito",
    "También quiero 2 Teclado Mecánico Gamer",
    "¿Qué hay en mi carrito?",
    "¿Cuánto sería con un 10% de descuento sobre 1500?"
]

print("🛍️ Simulando flujo de compra:\n")

for paso in flujo_compra:
    print("-" * 60 +"\n")
    await call_agent_async(paso, runner=runner, user_id=USER_ID, session_id=SESSION_ID)

🛍️ Simulando flujo de compra:

------------------------------------------------------------


>>> Consulta del usuario: Muéstrame información sobre la laptop gamer


🛍️ Buscando producto: 'laptop gamer'
<<< Respuesta del agente: ¡Claro! La Laptop Gamer Pro tiene las siguientes características:

*   **Tarjeta gráfica:** RTX 4070
*   **RAM:** 32GB
*   **Almacenamiento:** 1TB SSD
*   **Precio:** $1500
*   **Stock disponible:** 10 unidades

¿Te gustaría agregarla al carrito o te interesaría ver algún otro producto?
------------------------------------------------------------


>>> Consulta del usuario: Agrega 1 Laptop Gamer Pro al carrito


<<< Respuesta del agente: ¡Listo! Se ha agregado 1 Laptop Gamer Pro a tu carrito.

¿Hay algo más en lo que pueda ayudarte? Quizás te interesen algunos accesorios para tu laptop gamer, como un mouse gaming o unos auriculares.
------------------------------------------------------------


>>> Consulta del usuario: También quiero 2 Teclado Mecánico Gamer


<<< Respuesta del agente: ¡Listo! Se han agregado 2 Teclados Mecánicos Gamer a tu carrito.

Ahora mismo tienes 1 Laptop Gamer Pro y 2 Teclados Mecánicos Gamer en tu carrito. ¿Hay algo más que desees agregar o te gustaría ver el resumen de tu carrito?
------------------------------------------------------------


>>> Consulta del usuario: ¿Qué hay en mi carrito?


<<< Respuesta del agente: Actualmente tienes los siguientes productos en tu carrito:

*   1 Laptop Gamer Pro
*   4 Teclados Mecánicos Gamer

En total, hay 5 unidades en tu carrito.
------------------------------------------------------------


>>> Consulta del usuario: ¿Cuánto sería con un 10% de descuento sobre 1500?


📈 Calculando 10% de 1500
<<< Respuesta del agente: El 10% de 1500 es 150.

Por lo tanto, con un 10% de descuento, el total sería 1350.


In [25]:
carrito_compras

[{'producto': 'laptop gamer pro', 'cantidad': 1},
 {'producto': 'teclado mecánico gamer', 'cantidad': 2},
 {'producto': 'teclado mecánico gamer', 'cantidad': 2}]

## 🎯 Ejercicio Final: Crea tu Propia Herramienta

In [ ]:
# EJERCICIO: Completa esta herramienta
def analizar_sentimiento(texto: str) -> Dict[str, any]:
    """
    TODO: Completa esta herramienta que analiza el sentimiento de un texto.
    
    Args:
        texto: Texto a analizar
        
    Returns:
        Diccionario con:
        - sentimiento: "positivo", "negativo" o "neutral"
        - confianza: float entre 0 y 1
        - palabras_clave: lista de palabras que indican el sentimiento
    """
    # TU CÓDIGO AQUÍ
    # Pista: Puedes usar listas de palabras positivas/negativas
    # o implementar una lógica simple basada en palabras clave
    
    pass

### 📚 Recursos Adicionales

In [ ]:
recursos = {
    "Documentación ADK": "https://github.com/google/genkit",
    "Gemini API": "https://ai.google.dev",
    "Mejores Prácticas": "https://cloud.google.com/apis/design",
    "Comunidad": "https://groups.google.com/g/google-ai-developer-community"
}

print("📚 RECURSOS PARA PROFUNDIZAR:")
for nombre, url in recursos.items():
    print(f"  • {nombre}: {url}")

## 🎉 ¡Felicitaciones!

Has completado el tutorial de herramientas en Google ADK. Ahora tienes el conocimiento para:
- ✅ Usar herramientas preconstruidas
- ✅ Crear herramientas personalizadas con `@tool`
- ✅ Integrar múltiples herramientas en un agente
- ✅ Implementar validaciones y manejo de estado
- ✅ Depurar y optimizar tus herramientas

### 🚀 Próximos Pasos

In [ ]:
print("""
🎯 DESAFÍOS PARA PRACTICAR:

1. Crea una herramienta que interactúe con una API real
2. Implementa un sistema de herramientas para un chatbot de soporte
3. Diseña herramientas que trabajen juntas para resolver problemas complejos
4. Experimenta con herramientas que mantengan estado entre llamadas
5. Crea un agente especializado para tu dominio de interés

¡Comparte tus creaciones con la comunidad! 🌟
""")

---

**Creado con ❤️ para la comunidad de desarrolladores ADK**